# Entendendo um pouco do Google Colaboratory

In [0]:
!cat /proc/meminfo

In [0]:
!cat /proc/cpuinfo

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# Setup


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
import os
import math

TB_DIR = './Graph'

## Visualizaremos o treinamento utilizando o Tensorboard

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(TB_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Baixando e visualizando a base de dados

In [0]:
import tensorflow as tf

train, test = tf.keras.datasets.mnist.load_data()
mnist_x, mnist_y = train

mnist_ds = tf.data.Dataset.from_tensor_slices(mnist_x)
print(mnist_ds)


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

for i in range(1, 5):
  plt.subplot(220+i)
  plt.imshow(mnist_x[i], cmap=plt.get_cmap('gray'))
  print(mnist_y[i])

# Código

In [0]:
model_name = 'mnist_1.0'


def model_fn(features, labels, mode, params):
    tf.summary.image('image', features)
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    XX = tf.reshape(features, [-1, 784])

    # The model
    Y = tf.nn.softmax(tf.matmul(XX, W) + b)
    
    tf.summary.histogram(W.name, W)
    tf.summary.histogram(b.name, b)
        
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        # loss function
        cross_entropy = -tf.reduce_sum(tf.one_hot(labels, 10) * tf.log(Y))
        # % of correct answers found in batch
        predictions = tf.argmax(Y,1)
        accuracy = tf.metrics.accuracy(predictions, labels)

        evalmetrics = {"accuracy/mnist": accuracy}
        if mode == tf.estimator.ModeKeys.TRAIN:
            tf.summary.scalar("accuracy/mnist", accuracy[1])
            tf.summary.scalar("learning_rate", params["learning_rate"])
            optimizer = tf.train.GradientDescentOptimizer(params["learning_rate"])
            train_step = optimizer.minimize(cross_entropy,
                                            global_step=tf.train.get_global_step())
        else:
            train_step = None
    else:
        cross_entropy = None
        train_step = None
        evalmetrics = None

    return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={"classid": predictions},
            loss=cross_entropy,
            train_op=train_step,
            eval_metric_ops=evalmetrics)

In [0]:
def make_input_fn(mnist_x, mnist_y, batch_size, mode, shuffle=False):
    def _input_fn():
        ds = tf.data.Dataset.from_tensor_slices((
            tf.cast(tf.expand_dims(mnist_x, -1), tf.float32)/256.0, 
            tf.cast(mnist_y, tf.int32)))
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
        else:
            num_epochs = 1 # end-of-input after this
        if shuffle or mode == tf.estimator.ModeKeys.TRAIN:
            ds = ds.shuffle(5000).repeat(num_epochs).batch(batch_size)
        else:
            ds = ds.repeat(num_epochs).batch(batch_size)
        ds = ds.prefetch(2)
        return ds
    return _input_fn


def train_and_evaluate(model_dir, hparams):
    
    estimator = tf.estimator.Estimator(
        model_fn = model_fn,
        params = hparams,
        config= tf.estimator.RunConfig(
            save_checkpoints_steps = 1000,
            log_step_count_steps=500
            ),
        model_dir = model_dir)
    
    train, test = tf.keras.datasets.mnist.load_data()
    
    train_x, train_y = train
    train_spec = tf.estimator.TrainSpec(
        input_fn = make_input_fn(
            train_x, train_y,
            hparams['batch_size'],
            mode = tf.estimator.ModeKeys.TRAIN),
        max_steps = (50000//hparams["batch_size"]) * 20)
    
    test_x, test_y = test
    eval_spec = tf.estimator.EvalSpec(
        input_fn = make_input_fn(
            test_x, test_y,
            hparams['batch_size'],
            mode = tf.estimator.ModeKeys.EVAL
        ),
        start_delay_secs = 1,
        throttle_secs = 1
    )

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [0]:

params = {
    "batch_size": 100,
    "learning_rate": 0.003,
    "pkeep": 0.75
}

train_and_evaluate(os.path.join(TB_DIR, model_name), params)